# メトリクスを使った学習ジョブの監視
本ハンズオンではAmazon SageMaker を使用して学習した場合のメトリクスの監視について実施します。 前半ではAmazon CloudWatch を使用した監視を行います。 CloudWatch では、学習ジョブから出力された raw データを収集し、ほぼリアルタイムの読み取り可能なメトリクスに加工できます。

また後半では、Keras がバッチ毎に出力した学習ログを活用してTensorboardにて学習の状況を監視する方法を実践します。

## Amazon SageMaker Python SDK での学習メトリクスの定義
CloudWatch にメトリクスを送信するためには、標準出力を正規表現でパースし、Estimator の metric_definitions 引数に渡します。もし`train:error`と`validation:error`というメトリクスを監視したい場合には、下記のようになります。

```python 
estimator = Estimator(image_name=ImageName,
            role='SageMakerRole', train_instance_count=1,
            train_instance_type='ml.c4.xlarge',            
            k=10,
            metric_definitions=[
                   {'Name': 'train:error', 'Regex': 'Train_error=(.*?);'},
                   {'Name': 'validation:error', 'Regex': 'Valid_error=(.*?);'}
            ])
```

## cifar10 の学習ジョブの監視
今回の Keras のがどのようなメトリクスを出力しているか、以前の cifar10_keras_sm の学習ジョブを SageMaker のコンソールから探しましょう。 学習ジョブの詳細からモニタリング、ログの表示と進んでいくことで標準出力を確認することが出来ます。

確認したメトリクスをパースできるように metric_definitions を定義します。

In [ ]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': 'loss: (.*?) '},
    {'Name': 'train:accuracy', 'Regex': 'acc: (.*?) '},
    {'Name': 'validation:loss', 'Regex': 'val_loss: (.*?) '},
    {'Name': 'validation:accuracy', 'Regex': 'val_acc: (.*?) '}
]

`metric_definitions=metric_definitions` として、学習ジョブを実行します。  

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

role = get_execution_role()

dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')

In [ ]:
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_sample1_CloudWatch.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 5},
                       train_instance_count=1,
                       train_instance_type='ml.p3.2xlarge',
                       metric_definitions=metric_definitions)


estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

### View the job training metrics
学習中のメトリクスの遷移は [SageMaker のコンソール](https://console.aws.amazon.com/sagemaker/home) から確認することが出来ます。下記のセルに表示されるリンクもお使い下さい。

In [ ]:
from IPython.core.display import Markdown

link = 'https://console.aws.amazon.com/cloudwatch/home?region='+sagemaker_session.boto_region_name+'#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20'+estimator.latest_training_job.job_name
display(Markdown('コチラのリンクもお使い下さい。: [リンク]('+link+')'))

## TensorBoard を使った監視
次は TensorBoard を使ったメトリクスの監視を行います。

次に`cifar10-keras-sm.py` をアップデートして 学習ログを TensorBoard へ送信しましょう。変更点は下記2点です。

①`from keras.callbacks import TensorBoard` の追加

② ModelCheckpoint callback の後に TensorBoard callback の追加。

```python
callbacks.append(TensorBoard(log_dir=args.model_dir,update_freq='epoch'))
```

### TensorBoard を使った学習ジョブの実行

In [ ]:
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_sample1_TensorBoard.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 5},
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions)

In [ ]:
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

TensorBoard で実行された学習ジョブを見ます。まずは [TensorBoard](https://github.com/tensorflow/tensorboard) をインストールします。

In [ ]:
!pip install tensorboard

次に TensorBoard をノートブックインスタンス上で起動します。お使いのリージョンと、学習ログを保管したS3パスから、起動コマンドを作成します。

In [ ]:
from IPython.core.display import Markdown

link = 'AWS_REGION=\''+sagemaker_session.boto_region_name+'\' tensorboard --logdir ' + estimator.model_dir + ' --host localhost --port 6006'
display(Markdown('表示された TensorBoard の起動コマンドを次のセルで実行して下さい'))
display(Markdown('!'+link))

In [ ]:
!AWS_REGION='現在お使いのリージョン' tensorboard --logdir 'ログを保管したS3パス' --host localhost --port 6006

下記のリンクをご自身のお使いの値に合わせて変更し、ブラウザでアクセスして下さい。\
`https://https://ノートブックインスタンス名.notebook.お使いのリージョン名.sagemaker.aws/proxy/6006/`